In [1]:
from sleepmate.agent import *
from sleepmate.mi import *
from sleepmate.meta import *
goals = get_goals()

In [2]:
!rm -f /Users/cck197/Downloads/virtual_greg/memory.pkl

# Testing Switching Goals

In [3]:
x = X(SYSTEM_DESCRIPTION)

get_agent len(tools)=8


In [12]:
x_ = MetaX(x.agent_executor.memory)

In [15]:
goal = x_(); goal



> Entering new AgentExecutor chain...


> Entering new LLMChain chain...
Prompt after formatting:

    Answer the following questions as best you can. You have access to the following tools:

    - insomnia_severity_index: Use this when the conversation history is empty or human has yet to agree
    to the Insomnia Severity Index survey.
    - open_focus: Use this when the human has completed the Valued Living Questionnaire.
    - leaves_on_a_stream: Use this when the human has completed the Open Focus exercise.
    - valued_living: Use this when the human has recorded a SEEDS diary entry.
    - diary_probe: Use this when the human has completed the Insomnia Severity Index, but has yet to agree to doing a sleep diary.
    - diary_entry: Use this after the human has agreed to do a sleep diary.
    - daily_routine: Use this after the human has completed at least one sleep diary entry.
    - seeds_probe: Use this to set up a SEEDS diary after the human was shown a daily routine.
    - s

'insomnia_severity_index'

In [6]:
x = X(SYSTEM_DESCRIPTION, goals[goal])

get_agent len(tools)=8


In [ ]:
x("2")



> Entering new AgentExecutor chain...


# Goal Prediction

## Using an LLM directly

In [18]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
)
from langchain.chains import LLMChain
from langchain.memory import ReadOnlySharedMemory

In [ ]:
2. diary_probe Agreeing to a sleep diary is the next thing: diary_probe.
3. diary_entry If they agreed to the sleep diary, the next priorty is to record a diary entry.
4. open_focus After that, they should move on to the "open focus" exercise.

In [50]:
template = """
You are an AI clinician helping a human with insomnia. Given the conversation below, your job is to decide what do next.
1. insomnia_severity_index Performing the ISI comes first. If there's no mention of the ISI, it should happen next.
If the human hasn't done this exercise, they must do it first.
2. diary_probe If there's no mention of agreeing to do a sleep diary, it should come next.

Output the appropriate label only.
"""

In [51]:
prompt = ChatPromptTemplate(
        messages=[
            SystemMessagePromptTemplate.from_template(template),
            # The `variable_name` here is what must align with memory
            MessagesPlaceholder(variable_name="chat_history"),
            HumanMessagePromptTemplate.from_template("{input}"),
        ]
    )

In [53]:
llm = ChatOpenAI(model_name="gpt-4")
chain = LLMChain(llm=llm, prompt=prompt, memory=ReadOnlySharedMemory(memory=x.agent_executor.memory))

In [54]:
chain.run("next action:")



> Entering new LLMChain chain...
Prompt after formatting:
System: 
You are an AI clinician helping a human with insomnia. Given the conversation below, your job is to decide what do next.
1. insomnia_severity_index Performing the ISI comes first. If there's no mention of the ISI, it should happen next.
If the human hasn't done this exercise, they must do it first.
2. diary_probe If there's no mention of agreeing to do a sleep diary, it should come next.

Output the appropriate label only.

Human: yep
AI: Great! Is now a good time for you to record a sleep diary entry?
Human: next action:

> Finished chain.


'diary_probe'

## Using an agent

In [56]:
def get_isi(
    memory: ReadOnlySharedMemory, goal: str, utterance: str, model_name=model_name
) -> str:
    """Use this unless the human has yet to agree to the Insomnia Severity Index.
    """
    return "insomnia_severity_index"

In [59]:
x2 = X(SYSTEM_DESCRIPTION, "Your goal is to decide what action to perform next. Output the label only.", [get_isi])

get_agent len(tools)=1
load_memory filename='/Users/cck197/Downloads/virtual_greg/memory.pkl'


In [60]:
x2("next action:")



> Entering new AgentExecutor chain...
functions.get_isi

> Finished chain.
functions.get_isi


False

## Custom Agent

In [4]:
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.prompts import StringPromptTemplate
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.utilities import GoogleSearchAPIWrapper
from langchain.chains import LLMChain
from typing import List, Union
from langchain.schema import AgentAction, AgentFinish, OutputParserException
from langchain.agents import tool
from langchain.memory import ReadOnlySharedMemory
import re

In [5]:
goals.keys()

dict_keys(['insomnia_severity_index', 'diary_probe', 'diary_entry', 'open_focus', 'leaves_on_a_stream', 'valued_living', 'test', 'seeds_probe', 'seeds_entry', 'knowledge', 'daily_routine'])

In [20]:
def insomnia_severity_index(word: str) -> str:
    """Use this when the human has yet to agree to the Insomnia Severity Index survey."""
    return "insomnia_severity_index"

def open_focus(word: str) -> str:
    """Use this when the human has completed the Valued Living Questionnaire."""
    return "open_focus"

def leaves_on_a_stream(word: str) -> str:
    """Use this when the human has completed the Open Focus exercise."""
    return "leaves_on_a_stream"

def valued_living(word: str) -> str:
    """Use this when the human has recorded a SEEDS diary entry."""
    return "valued_living"

def diary_probe(word: str) -> str:
    """Use this when the human has completed the Insomnia Severity Index, but has yet to agree to doing a sleep diary."""
    return "diary_probe"

def diary_entry(word: str) -> str:
    """Use this after the human has agreed to do a sleep diary."""
    return "diary_entry"

def daily_routine(word: str) -> str:
    """Use this after the human has completed at least one sleep diary entry."""
    return "daily_routine"

def seeds_probe(word: str) -> str:
    """Use this to set up a SEEDS diary after the human was shown a daily routine."""
    return "seeds_probe"

def seeds_entry(word: str) -> str:
    """Use this to record a SEEDS diary entry after the human has defined their SEEDS."""
    return "seeds_entry"

tools = [Tool(name=t.__name__, func=t, description=t.__doc__, return_direct=True) for t in [
    insomnia_severity_index, open_focus, leaves_on_a_stream, valued_living, 
    diary_probe, diary_entry, daily_routine, seeds_probe, seeds_entry,]]

In [21]:
import random
random.shuffle(tools)

In [119]:
# Define which tools the agent can use to answer user queries
#search = GoogleSearchAPIWrapper()
tools = [
    #Tool(
    #    name="Search",
    #    func=search.run,
    #    description="Useful for when you need to answer questions about current events",
    #),
    Tool(
        name="get_isi",
        func=get_isi,
        description=get_isi.__doc__,
        return_direct=True
    ),
    Tool(
        name="get_open_focus",
        func=get_open_focus,
        description=get_isi.__doc__,
        return_direct=True
    ),
]

In [18]:
# Set up the base template
template = """Answer the following questions as best you can, but speaking as a pirate might speak. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Remember to speak as a pirate when giving your final answer. Use lots of "Arg"s

Question: {input}
{agent_scratchpad}"""

In [19]:
# Set up the base template
template_with_history = """Answer the following questions as best you can, but speaking as a pirate might speak. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Remember to speak as a pirate when giving your final answer. Use lots of "Arg"s

Previous conversation history:
{history}

New question: {input}
{agent_scratchpad}"""

In [22]:
# Set up the base template
template_with_history = """Answer the following questions as best you can. You have access to the following tools:

{tools}

The order of the tools is unimportant! Consider each tool carefully.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Previous conversation history:
{chat_history}

New question: {input}
{agent_scratchpad}"""

In [11]:
# Set up a prompt template
class CustomPromptTemplate(StringPromptTemplate):
    # The template to use
    template: str
    # The list of tools available
    tools: List[Tool]

    def format(self, **kwargs) -> str:
        # Get the intermediate steps (AgentAction, Observation tuples)
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts
        # Create a tools variable from the list of tools provided
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        # Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        return self.template.format(**kwargs)

In [10]:
prompt = CustomPromptTemplate(
    template=template,
    tools=tools,
    # This omits the `agent_scratchpad`, `tools`, and `tool_names` variables because those are generated dynamically
    # This includes the `intermediate_steps` variable because that is needed
    input_variables=["input", "intermediate_steps"]
)

NameError: name 'CustomPromptTemplate' is not defined

In [23]:
prompt_with_history = CustomPromptTemplate(
    template=template_with_history,
    tools=tools,
    # This omits the `agent_scratchpad`, `tools`, and `tool_names` variables because those are generated dynamically
    # This includes the `intermediate_steps` variable because that is needed
    input_variables=["input", "intermediate_steps", "chat_history"]
)

In [14]:
class CustomOutputParser(AgentOutputParser):

    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        # Check if agent should finish
        if "Final Answer:" in llm_output:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        # Parse out the action and action input
        regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            raise OutputParserException(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        # Return the action and action input
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)

In [24]:
output_parser = CustomOutputParser()

In [175]:
llm = OpenAI(temperature=0)

In [14]:
# LLM chain consisting of the LLM and a prompt
llm_chain = LLMChain(llm=llm, prompt=prompt)

In [25]:
llm_chain = LLMChain(llm=ChatOpenAI(temperature=0, model_name="gpt-4-0613"), prompt=prompt_with_history)

In [26]:
tool_names = [tool.name for tool in tools]
agent = LLMSingleActionAgent(
    llm_chain=llm_chain,
    output_parser=output_parser,
    stop=["\nObservation:"],
    allowed_tools=tool_names
)

In [42]:
from langchain.memory import ConversationBufferWindowMemory

In [43]:
memory=ConversationBufferWindowMemory(k=2)

In [102]:
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True)

In [27]:
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True, 
                                                    memory=ReadOnlySharedMemory(memory=x.agent_executor.memory))

In [26]:
agent_executor.run("How many people live in canada as of 2023?")



> Entering new AgentExecutor chain...


> Entering new LLMChain chain...
Prompt after formatting:
Answer the following questions as best you can, but speaking as a pirate might speak. You have access to the following tools:

Search: Useful for when you need to answer questions about current events

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Search]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Remember to speak as a pirate when giving your final answer. Use lots of "Arg"s

Previous conversation history:


New question: How many people live in canada as of 2023?


> Finished chain.
Thought: I need to find out the population of Canada in 2023
Action: Search
Ac

"Arrr, Canada be havin' 40 million people in 2023!"

In [28]:
agent_executor.run("what's the next action?")



> Entering new AgentExecutor chain...


> Entering new LLMChain chain...
Prompt after formatting:
Answer the following questions as best you can. You have access to the following tools:

diary_entry: Use this after the human has agreed to do a sleep diary.
leaves_on_a_stream: Use this when the human has completed the Open Focus exercise.
seeds_entry: Use this to record a SEEDS diary entry after the human has defined their SEEDS.
insomnia_severity_index: Use this when the human has yet to agree to the Insomnia Severity Index survey.
valued_living: Use this when the human has recorded a SEEDS diary entry.
daily_routine: Use this after the human has completed at least one sleep diary entry.
open_focus: Use this when the human has completed the Valued Living Questionnaire.
diary_probe: Use this when the human has completed the Insomnia Severity Index, but has yet to agree to doing a sleep diary.
seeds_probe: Use this to set up a SEEDS diary after the human was shown a daily routine.

The

'open_focus'